# Easy MD Analysis Example Notebook

In [1]:
from EMDA.emda import EMDA

/home/mcanyelles/miniconda3/envs/rcbs/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/mcanyelles/miniconda3/envs/rcbs/lib/python3.11/site-packages/MDAnalysis/topology/TPRParser.py:161: DeprecationWarning: 'xdrlib' is deprecated and slated for removal in Python 3.13
  import xdrlib


In [2]:
parameters = 'parameters.prmtop'
trajectory = 'trajectory.nc'

In [3]:
emda = EMDA(parameters=parameters, trajectory=trajectory)

Trajectory has been loaded!


In [6]:
emda.select('C10', 'C10', sel_type='at_name')
emda.select('C11', 'C11', sel_type='at_name')
emda.select('C12', 'C12', sel_type='at_name')
emda.select('C13', 'C13', sel_type='at_name')
emda.select('C14', 'C14', sel_type='at_name')

emda.select('H12', ['H12A', 'H12B'], sel_type='at_name')
emda.select('Fe', 10596, sel_type='at_num')

emda.select('cof', 10597, sel_type='at_num')

emda.select('COO', [10599, 10600, 10601], sel_type='at_num')


emda.select('subs', 666, sel_type='res_num')

emda.select('protein', 'protein')


In [7]:
emda.print_available_adders()

Method: add_RMSD
Help:
DESCRIPTION:
    This function outputs the RMSD of a selection

No usage available.


Method: add_angle
Help:
DESCRIPTION:
    This functions measures the angle between 3 specified atoms and returns the value between 0 and 360 degrees.
    The input selections have to be single atoms.

No usage available.


Method: add_contacts
Help:
DESCRIPTION:
    This function takes a Universe, a selection and a radius and returns the list of residues nearer than the specified radius.

No usage available.


Method: add_dihedral
Help:
DESCRIPTION:
    This functions measures the dihedral angle between 4 specified atoms and returns the dihedral value between 0 and 360 degrees.
    The input selections have to be single atoms.

No usage available.


Method: add_distWATbridge
Help:
DESCRIPTION
    This function takes a Universe, two selections and the size of their environments and returns the nearest bridging water between the two selections and the distance to both of them.

No

In [8]:
emda.add_distance('dist_H12_cof', 'H12', 'cof', type='min')
emda.add_angle('ang_C12_cof_Fe', 'Fe', 'cof', 'C12')
emda.add_dihedral('dihe_C10C14', 'C10', 'C11', 'C13', 'C14', domain=180)
emda.add_contacts('contacts_COO', 'COO', sel_env=5, interactions='all', include_WAT=True)
emda.add_contacts('contacts_protein', 'protein')
emda.add_pKa('pka_protein', 'protein', )
emda.add_RMSD('rmsd_subs', 'subs', )

In [9]:
emda.run()

{'contacts_COO', 'contacts_protein', 'dihe_C10C14', 'dist_H12_cof', 'ang_C12_cof_Fe'}


Measuring: 100%|██████████| 10/10 [00:17<00:00,  1.70s/Frame]


In [10]:
print(emda.measures)

{'dist_H12_cof': Measure dataclass with:
	Name: dist_H12_cof
	Type: distance
	Sel: [<AtomGroup with 2 atoms>, <AtomGroup with 1 atom>]
	Status: Calculated
, 'ang_C12_cof_Fe': Measure dataclass with:
	Name: ang_C12_cof_Fe
	Type: angle
	Sel: [<AtomGroup with 1 atom>, <AtomGroup with 1 atom>, <AtomGroup with 1 atom>]
	Status: Calculated
, 'dihe_C10C14': Measure dataclass with:
	Name: dihe_C10C14
	Type: dihedral
	Sel: [<AtomGroup with 1 atom>, <AtomGroup with 1 atom>, <AtomGroup with 1 atom>, <AtomGroup with 1 atom>]
	Status: Calculated
, 'contacts_COO': Measure dataclass with:
	Name: contacts_COO
	Type: contacts
	Sel: [<AtomGroup with 3 atoms>, <AtomGroup with 77 atoms, with selection 'around 5 group select' on the entire Universe.>]
	Status: Calculated
, 'contacts_protein': Measure dataclass with:
	Name: contacts_protein
	Type: contacts
	Sel: [<AtomGroup with 10510 atoms>, 3]
	Status: Calculated
, 'pka_protein': Measure dataclass with:
	Name: pka_protein
	Type: pka
	Sel: [<AtomGroup with